In [1]:
import pandas as pd
import numpy as np
import requests #for HTTP requests
import xlsxwriter #for writing the excel sheet
import math

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv') #reading the data
stocks.head(5)

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV


In [3]:
#using IEX Cloud API token
#The data is randomly generated using sandbox API token.
from secrets import IEX_CLOUD_API_TOKEN

In [4]:
#making first API call
#we need market capitalization of each stock and price of each stock.
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote/?token={IEX_CLOUD_API_TOKEN}'

#doing the http request
data = requests.get(api_url).json()
print(data)

{'symbol': 'AAPL', 'companyName': 'Apple Inc', 'primaryExchange': 'GAKNSTELEQLRM SNB CA)OEALT /AGS(D', 'calculationPrice': 'close', 'open': 131.91, 'openTime': 1698773322210, 'openSource': 'ifialocf', 'close': 136.35, 'closeTime': 1644652717970, 'closeSource': 'offiicla', 'high': 136.105, 'highTime': 1640758580713, 'highSource': ' dayemide1incrpl5uee t ', 'low': 129.802, 'lowTime': 1683661754070, 'lowSource': ' c mde5ied1plu naereyti', 'latestPrice': 136.1, 'latestSource': 'Close', 'latestTime': 'May 7, 2021', 'latestUpdate': 1667608656777, 'latestVolume': 82823130, 'iexRealtimePrice': None, 'iexRealtimeSize': None, 'iexLastUpdated': None, 'delayedPrice': 135.8, 'delayedPriceTime': 1654474291366, 'oddLotDelayedPrice': 133.582, 'oddLotDelayedPriceTime': 1693481334483, 'extendedPrice': 132.84, 'extendedChange': 0.29, 'extendedChangePercent': 0.00226, 'extendedPriceTime': 1633775970507, 'previousClose': 134.18, 'previousVolume': 81995311, 'change': 0.7, 'changePercent': 0.00549, 'volume':

In [5]:
#parsing the data
price = data['latestPrice']
market_cap = data['marketCap']

In [6]:
my_columns = ['Ticker', 'Price', 'Market Capitalization', 'Number of Shares to Buy']
final_dataframe = pd.DataFrame(columns = my_columns)
final_dataframe

,Ticker,Price,Market Capitalization,Number of Shares to Buy


In [7]:
final_dataframe = final_dataframe.append(
pd.Series([
    symbol,
    price,
    market_cap,
    'N/A'
],
index = my_columns
),
ignore_index=True
)
final_dataframe

,Ticker,Price,Market Capitalization,Number of Shares to Buy
0,AAPL,136.1,2227604167882,N/A


In [8]:
#Looping through tickers with one API call per ticker
final_dataframe = pd.DataFrame(columns = my_columns)
for symbol in stocks['Ticker'][:5]:
    api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote/?token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(api_url).json()
    final_dataframe = final_dataframe.append(
        pd.Series([
            symbol,
            price,
            market_cap,
            'N/A'
        ],
        index = my_columns
        ),
        ignore_index=True
        )
final_dataframe

,Ticker,Price,Market Capitalization,Number of Shares to Buy
0,A,136.1,2227604167882,N/A
1,AAL,136.1,2227604167882,N/A
2,AAP,136.1,2227604167882,N/A
3,AAPL,136.1,2227604167882,N/A
4,ABBV,136.1,2227604167882,N/A


In [9]:
#Using batch API to make a single request for 100 stocks at a time.
#func to divide all stocks into grps of 100
def chunks(lst, n):
    for i in range(0, len(lst), n):
        yield lst[i: i+n]

In [10]:
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))  #to join the list of 100 stocks by comma
#    print(symbol_strings)

final_dataframe = pd.DataFrame(columns = my_columns)

for sym in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={sym}&types=quote&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in sym.split(','):
        final_dataframe = final_dataframe.append(
            pd.Series([
                symbol,
                data[symbol]['quote']['latestPrice'],
                data[symbol]['quote']['marketCap'],
                'N/A'
            ],
                index = my_columns
            ),
            ignore_index = True
        )
        
final_dataframe

,Ticker,Price,Market Capitalization,Number of Shares to Buy
0,A,135.20,42045905671,N/A
1,AAL,23.00,14378840936,N/A
2,AAP,214.17,13726732537,N/A
3,AAPL,134.21,2270263743831,N/A
4,ABBV,116.60,205593020809,N/A
...,...,...,...,...
500,YUM,123.13,36160046886,N/A
501,ZBH,174.07,37454909127,N/A
502,ZBRA,510.02,27052531022,N/A
503,ZION,59.68,9797602127,N/A


In [11]:
# calculating the no. of shares to be bought of each company
portfolio_size = input("Enter the value:")

try:
    val = float(portfolio_size)
except ValueError:
    print("Enter float value:")
    portfolio_size = input("Enter the value:")

Enter the value: 1000000


In [12]:
position_size = float(portfolio_size) / len(final_dataframe.index)
for i in range(0, len(final_dataframe['Ticker'])-1):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / final_dataframe['Price'][i])

final_dataframe

,Ticker,Price,Market Capitalization,Number of Shares to Buy
0,A,135.20,42045905671,14
1,AAL,23.00,14378840936,86
2,AAP,214.17,13726732537,9
3,AAPL,134.21,2270263743831,14
4,ABBV,116.60,205593020809,16
...,...,...,...,...
500,YUM,123.13,36160046886,16
501,ZBH,174.07,37454909127,11
502,ZBRA,510.02,27052531022,3
503,ZION,59.68,9797602127,33


In [13]:
writer = pd.ExcelWriter('recommended_trades.xlsx', engine='xlsxwriter')
final_dataframe.to_excel(writer, sheet_name='Recommended Trades', index = False)

In [14]:
background_color = '#FFFF99'
font_color = '000000'
font_red = 'ff0000'

string_format = writer.book.add_format(
{
    'font_color': font_color,
    'bg_color': background_color,
    'border': 1
})

dollar_format = writer.book.add_format(
{
    'num_format': '$0.00',
    'font_color': font_color,
    'bg_color': background_color,
    'border': 1
})

integer_format = writer.book.add_format(
{
    'num_format': '0',
    'font_color': font_red,
    'bg_color': background_color,
    'border': 1
})

In [15]:
column_formats = {
    'A': ['Tickers', string_format],
    'B': ['Price', dollar_format],
    'C': ['Market Capitalization', dollar_format],
    'D': ['Number of Shares to Buy', integer_format]
}

for column in column_formats.keys():
    writer.sheets['Recommended Trades'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Recommended Trades'].write(f'{column}1', column_formats[column][0], string_format)

In [16]:
writer.save()